# Hello World Notebook

A minimal Jupyter notebook demonstrating basic Python concepts.

## 1. Print Hello World

In [ ]:
print("Hello, World!")

## 2. Store and Display a Greeting

In [ ]:
greeting = "Welcome to Jupyter Notebooks!"
print(greeting)

## 3. Create a Simple Function

In [ ]:
def greet(name):
    return f"Hello, {name}!"

# Call the function
message = greet("Python")
print(message)

## 4. Install Required Packages

In [ ]:
%pip install azure-identity azure-mgmt-resource

## 5. Verify Azure Connectivity

In [ ]:
from azure.identity import AzureCliCredential
from azure.mgmt.resource import SubscriptionClient

try:
    # Get credential from Azure CLI login
    credential = AzureCliCredential()
    
    # Get subscription information to verify identity
    subscription_client = SubscriptionClient(credential)
    subscription = next(subscription_client.subscriptions.list())
    
    print("✓ Successfully authenticated!")
    print(f"Subscription ID: {subscription.subscription_id}")
    print(f"Subscription Name: {subscription.display_name}")
    print(f"Tenant ID: {subscription.tenant_id}")
    
except Exception as e:
    print(f"✗ Not authenticated or error occurred: {e}")
    print("Please run 'az login' in the terminal first.")

## 6. Configuration

In [ ]:
# Azure Resource Configuration
BASE_NAME = "aigwpg"
RESOURCE_GROUP_NAME = f"{BASE_NAME}-rg"
LOCATION = "Sweden Central"

## 7. Define Helper Functions

In [ ]:
import subprocess
import json
import os

def terraform_up(tf_dir, tf_vars=None, output_vars=None):
    """
    Run terraform init, apply and capture outputs.
    
    Args:
        tf_dir: Path to terraform directory (relative to workspace root)
        tf_vars: Dictionary of Terraform variables to pass (optional)
        output_vars: List of output variable names to capture (optional)
    
    Returns:
        Dictionary with captured output values, or None if no output_vars specified
    """
    # Write variables to tfvars file if provided
    if tf_vars:
        tfvars_path = os.path.join(tf_dir, 'terraform.auto.tfvars.json')
        with open(tfvars_path, 'w') as f:
            json.dump(tf_vars, f, indent=2)
        print(f"✓ Created {tfvars_path}")
        print(f"  Variables: {tf_vars}")
    
    # Run terraform init
    print(f"\n> terraform init (in {tf_dir})")
    result = subprocess.run(
        ['terraform', 'init'],
        cwd=tf_dir
    )
    if result.returncode != 0:
        raise Exception(f"Terraform init failed with exit code {result.returncode}")
    
    # Run terraform apply
    print(f"\n> terraform apply -auto-approve (in {tf_dir})")
    result = subprocess.run(
        ['terraform', 'apply', '-auto-approve'],
        cwd=tf_dir
    )
    if result.returncode != 0:
        raise Exception(f"Terraform apply failed with exit code {result.returncode}")
    
    # Capture outputs if requested
    if output_vars:
        print(f"\n> terraform output -json (in {tf_dir})")
        result = subprocess.run(
            ['terraform', 'output', '-json'],
            cwd=tf_dir,
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            outputs = json.loads(result.stdout)
            
            # Extract requested variables
            captured_vars = {}
            for var in output_vars:
                if var in outputs:
                    captured_vars[var] = outputs[var]['value']
                else:
                    print(f"⚠ Warning: Output variable '{var}' not found")
            
            print("✓ Terraform outputs captured successfully!")
            for var, value in captured_vars.items():
                print(f"  {var}: {value}")
            
            return captured_vars
        else:
            raise Exception(f"Terraform output failed: {result.stderr}")
    
    return None

## 8. Deploy Azure Resource Group with Terraform

In [ ]:
# Deploy resource group and capture outputs
outputs = terraform_up(
    tf_dir='terraform/01_rg',
    tf_vars={
        'resource_group_name': RESOURCE_GROUP_NAME,
        'location': LOCATION
    },
    output_vars=['resource_group_id', 'resource_group_location', 'resource_group_name']
)

# Extract individual variables for convenience
resource_group_id = outputs['resource_group_id']
resource_group_location = outputs['resource_group_location']
resource_group_name = outputs['resource_group_name']

## 9. Clean Terraform State

In [ ]:
import shutil
import os
import glob

tf_dir = 'terraform/01_rg'

# Remove .terraform directory
terraform_dir = os.path.join(tf_dir, '.terraform')
if os.path.exists(terraform_dir):
    shutil.rmtree(terraform_dir)
    print(f"✓ Deleted {terraform_dir}")
else:
    print(f"  {terraform_dir} does not exist")

# Remove tfstate files
state_files = glob.glob(os.path.join(tf_dir, '*.tfstate*'))
for file in state_files:
    os.remove(file)
    print(f"✓ Deleted {file}")

if not state_files:
    print(f"  No tfstate files found in {tf_dir}")

# Remove auto-generated tfvars file
tfvars_file = os.path.join(tf_dir, 'terraform.auto.tfvars.json')
if os.path.exists(tfvars_file):
    os.remove(tfvars_file)
    print(f"✓ Deleted {tfvars_file}")

print("\n✓ Cleanup complete!")